In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
import geopandas as gpd

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import plotly.express as px
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

Mounted at /content/drive


In [2]:
# data preprocess
gdf_3 = gpd.read_file(f'/content/drive/My Drive/Mantises!/data/gadm41_USA_shp/gadm41_USA_2.shp')
gdf_3 = gdf_3[(gdf_3['NAME_1'] == 'California') & (gdf_3['TYPE_2'] == 'County')]

df = pd.read_csv(f'/content/drive/My Drive/Mantises!/data/mantodea_ca_gbif_simple.csv', sep='\t')

In [3]:
species_df = df[['species', 'decimalLatitude', 'decimalLongitude', 'elevation', 'depth']]

In [4]:
from shapely.geometry import Point

# Create a new geometry column in your pandas dataframe
geometry = [Point(xy) for xy in zip(species_df.decimalLongitude, species_df.decimalLatitude)]
geo_species_df = gpd.GeoDataFrame(species_df, geometry=geometry)


In [5]:
geo_species_df.set_crs(epsg=4326, inplace=True)

gdf_3.set_crs(epsg=4326, inplace=True)
# If necessary, transform one of the dataframes to match the other's CRS
geo_species_df = geo_species_df.to_crs(gdf_3.crs)

In [6]:
joined = gpd.sjoin(geo_species_df, gdf_3, how="inner", op="intersects")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
joined.head()

,species,decimalLatitude,decimalLongitude,elevation,depth,geometry,index_right,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2
0,Mantis religiosa,38.569964,-122.689257,NaN,NaN,POINT (-122.68926 38.56996),231,USA.5.49_1,USA,United States,USA.5_1,California,NA,Sonoma,NA,NA,County,County,NA,US.CA.SO
2,Mantis religiosa,38.569659,-122.689175,NaN,NaN,POINT (-122.68918 38.56966),231,USA.5.49_1,USA,United States,USA.5_1,California,NA,Sonoma,NA,NA,County,County,NA,US.CA.SO
9,Mantis religiosa,38.624724,-122.871580,NaN,NaN,POINT (-122.87158 38.62472),231,USA.5.49_1,USA,United States,USA.5_1,California,NA,Sonoma,NA,NA,County,County,NA,US.CA.SO
10,Mantis religiosa,38.575230,-122.693980,NaN,NaN,POINT (-122.69398 38.57523),231,USA.5.49_1,USA,United States,USA.5_1,California,NA,Sonoma,NA,NA,County,County,NA,US.CA.SO
11,Mantis religiosa,38.584350,-122.699930,NaN,NaN,POINT (-122.69993 38.58435),231,USA.5.49_1,USA,United States,USA.5_1,California,NA,Sonoma,NA,NA,County,County,NA,US.CA.SO


In [8]:
population_df = pd.read_csv(f'/content/drive/My Drive/Mantises!/data/ACSDT5Y2020.B01003-Data.csv')

In [9]:
county_size_df = pd.read_csv(f'/content/drive/My Drive/Mantises!/data/California_Counties_Area.csv')

In [10]:
county_density_df = pd.merge(county_size_df, population_df, on='County')

In [11]:
county_density_df['human_population_density'] = county_density_df['Population Total'] / county_density_df['Land Area (sq mi)']

In [12]:
joined = joined.rename(columns = {'NAME_2': 'County'})

In [13]:
joined = joined[['species', 'decimalLatitude', 'decimalLongitude', 'elevation', 'depth', 'geometry', 'County']]

In [14]:
all_data = pd.merge(joined, county_density_df, on='County')

In [15]:
all_data.head()

,species,decimalLatitude,decimalLongitude,elevation,depth,geometry,County,Rank,Land Area (sq mi),Population Total,human_population_density
0,Mantis religiosa,38.569964,-122.689257,NaN,NaN,POINT (-122.68926 38.56996),Sonoma,29,1575.63,496801,315.303085
1,Mantis religiosa,38.569659,-122.689175,NaN,NaN,POINT (-122.68918 38.56966),Sonoma,29,1575.63,496801,315.303085
2,Mantis religiosa,38.624724,-122.871580,NaN,NaN,POINT (-122.87158 38.62472),Sonoma,29,1575.63,496801,315.303085
3,Mantis religiosa,38.575230,-122.693980,NaN,NaN,POINT (-122.69398 38.57523),Sonoma,29,1575.63,496801,315.303085
4,Mantis religiosa,38.584350,-122.699930,NaN,NaN,POINT (-122.69993 38.58435),Sonoma,29,1575.63,496801,315.303085


In [16]:
all_data = all_data.drop('Rank', axis=1)

In [17]:
all_data.columns
status = {
    'Mantis religiosa': 'introduced',
    'Stagmomantis californica': 'native',
    'Litaneutria pacifica': 'native',
    'Litaneutria skinneri': 'native',
    'Stagmomantis limbata': 'native',
    'Iris oratoria': 'introduced',
    'Miomantis caffra': 'introduced',
    'Litaneutria ocularis': 'native',
    'Tenodera sinensis': 'introduced',
    'Litaneutria chaparrali': 'native',
    'Hierodula patellifera': 'introduced',
    'Litaneutria minor': 'native',
    'Yersiniops newboldi': 'native',
    'Thesprotia graminis': 'native'
}


all_data['native'] = all_data['species'].map(status)

In [18]:
all_data.head()

,species,decimalLatitude,decimalLongitude,elevation,depth,geometry,County,Land Area (sq mi),Population Total,human_population_density,native
0,Mantis religiosa,38.569964,-122.689257,NaN,NaN,POINT (-122.68926 38.56996),Sonoma,1575.63,496801,315.303085,introduced
1,Mantis religiosa,38.569659,-122.689175,NaN,NaN,POINT (-122.68918 38.56966),Sonoma,1575.63,496801,315.303085,introduced
2,Mantis religiosa,38.624724,-122.871580,NaN,NaN,POINT (-122.87158 38.62472),Sonoma,1575.63,496801,315.303085,introduced
3,Mantis religiosa,38.575230,-122.693980,NaN,NaN,POINT (-122.69398 38.57523),Sonoma,1575.63,496801,315.303085,introduced
4,Mantis religiosa,38.584350,-122.699930,NaN,NaN,POINT (-122.69993 38.58435),Sonoma,1575.63,496801,315.303085,introduced


In [19]:
# getting climate data
climate_data = pd.read_csv(f'/content/drive/My Drive/Mantises!/output/climatic_data/climatic_data_20231206_050035.csv')

In [20]:
climate_data.head()

,State,County,Year,Month,prcp,tmax,tmin
0,California,Alameda,1980,1,115.634125,NaN,NaN
1,California,Alpine,1980,1,361.815216,NaN,NaN
2,California,Amador,1980,1,320.292175,NaN,NaN
3,California,Butte,1980,1,286.148041,NaN,NaN
4,California,Calaveras,1980,1,332.901245,NaN,NaN


In [21]:
climate_data = climate_data[['County', 'prcp', 'tmax', 'tmin']]
climate_data = climate_data.groupby('County').mean()

In [22]:
climate_data.head()

,prcp,tmax,tmin
County,,,
Alameda,40.319967,20.904928,9.695275
Alpine,81.807864,12.457218,-0.429198
Amador,80.838894,20.845305,7.341989
Butte,96.290673,22.181397,8.452543
Calaveras,78.762067,21.392718,7.554954


In [23]:
all_data = pd.merge(all_data, climate_data, on='County')

In [24]:
all_data.head()

,species,decimalLatitude,decimalLongitude,elevation,depth,geometry,County,Land Area (sq mi),Population Total,human_population_density,native,prcp,tmax,tmin
0,Mantis religiosa,38.569964,-122.689257,NaN,NaN,POINT (-122.68926 38.56996),Sonoma,1575.63,496801,315.303085,introduced,93.391093,20.492571,7.783081
1,Mantis religiosa,38.569659,-122.689175,NaN,NaN,POINT (-122.68918 38.56966),Sonoma,1575.63,496801,315.303085,introduced,93.391093,20.492571,7.783081
2,Mantis religiosa,38.624724,-122.871580,NaN,NaN,POINT (-122.87158 38.62472),Sonoma,1575.63,496801,315.303085,introduced,93.391093,20.492571,7.783081
3,Mantis religiosa,38.575230,-122.693980,NaN,NaN,POINT (-122.69398 38.57523),Sonoma,1575.63,496801,315.303085,introduced,93.391093,20.492571,7.783081
4,Mantis religiosa,38.584350,-122.699930,NaN,NaN,POINT (-122.69993 38.58435),Sonoma,1575.63,496801,315.303085,introduced,93.391093,20.492571,7.783081


In [25]:
features = ['decimalLatitude', 'decimalLongitude', 'human_population_density', 'prcp', 'tmin', 'tmax']
label = 'native'


In [25]:
all_data.to_csv('/content/drive/My Drive/Mantises!/output/all_data.csv')

In [28]:
c_range = [10e-12, 10e-8, 10e-4, 10e-1, 1, 10, 100, 1000, 10e4]
gamma_range = [10e-12, 10e-8, 10e-4, 10e-1, 1, 10, 100, 1000, 10e4, 10e5, 10e6]

no_na_data = all_data[features + [label]].dropna()





from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(no_na_data, test_size=0.2, random_state=42)




train_accuracy_holder = {'Label': [], 'Gamma': [], 'C': [], 'Train Accuracy': []}

for c in c_range:
  for gamma in gamma_range:
    # default kernel is rbf (Gaussian) kernel

    clf= SVC(kernel='linear',gamma=gamma, C=c)
    clf.fit(train_df[features], train_df[label])
    predictions = clf.predict(train_df[features])

    train_accuracy = accuracy_score(predictions, train_df[label])


    train_accuracy_holder['Gamma'].append(gamma)
    train_accuracy_holder['C'].append(c)
    train_accuracy_holder['Train Accuracy'].append(train_accuracy)

    print(f'Gamma: {gamma}, C: {c}, Train Accuracy: {train_accuracy}')


train_accuracy_df = pd.DataFrame(train_accuracy_holder)
train_accuracy_df.head()

Gamma: 1e-11, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 1e-07, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 0.001, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 1.0, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 1, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 10, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 100, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 1000, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 100000.0, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 1000000.0, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 10000000.0, C: 1e-11, Train Accuracy: 0.5061936936936937
Gamma: 1e-11, C: 1e-07, Train Accuracy: 0.6204954954954955
Gamma: 1e-07, C: 1e-07, Train Accuracy: 0.6204954954954955
Gamma: 0.001, C: 1e-07, Train Accuracy: 0.6204954954954955
Gamma: 1.0, C: 1e-07, Train Accuracy: 0.6204954954954955
Gamma: 1, C: 1e-07, Train Accuracy: 0.6204954954954955


KeyboardInterrupt: ignored

In [ ]:
# repeat same process for linear kernel!

c_range = [10e-12, 10e-8, 10e-4, 10e-1, 1, 10, 100]
no_na_data = all_data[features + [label]].dropna()





from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(no_na_data, test_size=0.2, random_state=42)




train_accuracy_holder = { 'C': [], 'Train Accuracy': []}

for c in c_range:

  print(c)
  clf= SVC(kernel='linear', C=c)
  clf.fit(train_df[features], train_df[label])
  predictions = clf.predict(train_df[features])

  train_accuracy = accuracy_score(predictions, train_df[label])

  train_accuracy_holder['C'].append(c)
  train_accuracy_holder['Train Accuracy'].append(train_accuracy)
  print(train_accuracy)


train_accuracy_df = pd.DataFrame(train_accuracy_holder)
train_accuracy_df.head()

1e-11
0.5061936936936937
1e-07
0.6204954954954955
0.001
0.7238175675675675
1.0
